# Building an AI Book Research Assistant

This notebook provides step-by-step instructions for replicating what Andrew did in the demo video on `Jupyter AI - AI Coding in Notebooks` short course on `Deeplearning.ai`. Please follow along, and feel free to play with different variations too! At the end, there are optional instructions for building a more advanced book research assistant than Andrew showed. 

*Useful Resources*:

- **AISuite GitHub Repo**: [https://github.com/andrewyng/aisuite](https://github.com/andrewyng/aisuite)
- **Open Library API**:
  - [Open Library](https://openlibrary.org/developers/api) offers free and public Web APIs for accessing book and author catalog data (no API key is required)
  - See `openlibrary_api_docs.md` in this lesson's directory for more information

## Step 1: Use Open Library API's Documentation

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Chat interface, create a new chat by clicking `+Chat` (choose any name for the file)
- In the chat window, attach the Markdown file `openlibrary_api_docs.md` using the `@` symbol:
  - Type `@`, then select `file` from the autocomplete menu
  - You'll see a list of available files in the lesson's directory, choose `openlibrary_api_docs.md` 
- Use a prompt like this:
   > Write a code cell that uses Open Library's search endpoint to look up books based on a user's query and returns a response listing identified books, with these fields for each book: key, title, author_name, first_publish_year, isbn, cover_id, edition_count, language, subject. Use a variable called limit, set to 5, to specify the number of returned results. Print the results. Use the Open Library API documentation in the attached file.
- Transfer the generated code to the cell below and run it

In [1]:
import requests

# User query
query = input("Enter a search query: ")

# Limit number of results
limit = 5

# API endpoint and parameters
url = "https://openlibrary.org/search.json"
params = {
    "q": query,
    "fields": "key,title,author_name,first_publish_year,isbn,cover_i,edition_count,language,subject",
    "limit": limit
}

# Send request
response = requests.get(url, params=params)
data = response.json()

# Extract and print results
books = data.get("docs", [])

print(f"\nFound {len(books)} results for query '{query}':\n")
for i, book in enumerate(books, start=1):
    print(f"Book {i}:")
    print(f"  Key: {book.get('key', 'N/A')}")
    print(f"  Title: {book.get('title', 'N/A')}")
    print(f"  Authors: {', '.join(book.get('author_name', ['N/A']))}")
    print(f"  First Published: {book.get('first_publish_year', 'N/A')}")
    print(f"  ISBNs: {', '.join(book.get('isbn', [])[:3]) if 'isbn' in book else 'N/A'}")
    print(f"  Cover ID: {book.get('cover_i', 'N/A')}")
    print(f"  Edition Count: {book.get('edition_count', 'N/A')}")
    print(f"  Languages: {', '.join(book.get('language', ['N/A']))}")
    print(f"  Subjects: {', '.join(book.get('subject', [])[:5]) if 'subject' in book else 'N/A'}")
    print()

Enter a search query:  science fiction



Found 5 results for query 'science fiction':

Book 1:
  Key: /works/OL1960472W
  Title: The Science Fiction Hall of Fame -- Volume One
  Authors: Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury
  First Published: 1970
  ISBNs: 0765305372, 9780575004627, 1250303621
  Cover ID: 6941459
  Edition Count: 31
  Languages: eng
  Subjects: Science fiction, American Science fiction, English Science fiction, Science fiction, american, Science fiction, english

Book 2:
  Key: /works/OL46125W
  Title: Foundation
  Authors: Isaac Asimov
  First Published: 1951
  ISBNs: 6053757373, 9780345317988, 1400000823
  Cover ID: 14612610
  Edition Count: 97
  Languages: ger, eng, kor, por, spa, rus, chi, fre, ita
  Subjects: Psychohistory, Open Library Staff Picks, Life on other planets, Fiction, Science Fiction

Book 3:
  Key: /works/OL59688W
  Title: Stranger in a Strange Land
  Authors: Robert A. Heinlein
  First Published: 1961
  ISBNs: 9788423320653, 9780786144068, 14

## Step 2: Wrap the API Call in a Tool Function

- In the chat interface, use a prompt like this:
  > Wrap the code above in a function `search_books`, with good docstring.
  > 
  > - Input arguments: `query` and `limit` (with default value of 10)
  > - Output: JSON string with:
  >   - `num_found`: total results
  >   - `books`: list of dicts with title, authors, first_publish_year, key, edition_count

In [ ]:
import requests
import json

def search_books(query: str, limit: int = 10) -> str:
    """
    Search for books using the Open Library Search API.

    Parameters
    ----------
    query : str
        The search query for finding books (e.g., title, author, subject, etc.).
    limit : int, optional
        The maximum number of book results to return. Default is 10.

    Returns
    -------
    str
        A JSON-formatted string containing the following structure:
        {
            "num_found": <total number of results found>,
            "books": [
                {
                    "title": <book title>,
                    "authors": <list of author names>,
                    "first_publish_year": <year of first publication>,
                    "key": <Open Library key>,
                    "edition_count": <number of editions>
                },
                ...
            ]
        }
    """
    print(f"📘 search_books called with query='{query}' and limit={limit}")
    print("🔎 Sending request to Open Library API...")

    # API endpoint and query parameters
    url = "https://openlibrary.org/search.json"
    params = {
        "q": query,
        "fields": "key,title,author_name,first_publish_year,edition_count",
        "limit": limit
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        print("✅ Request successful, processing response...")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return json.dumps({"error": str(e)}, indent=2)

    # Parse JSON response
    data = response.json()
    num_found = data.get("numFound", 0)
    print(f"📄 Found {num_found} total results.")

    # Extract relevant fields
    books = [
        {
            "title": book.get("title"),
            "authors": book.get("author_name", []),
            "first_publish_year": book.get("first_publish_year"),
            "key": book.get("key"),
            "edition_count": book.get("edition_count")
        }
        for book in data.get("docs", [])
    ]

    print(f"📚 Returning top {len(books)} book results.\n")

    # Prepare and return JSON response
    result = {
        "num_found": num_found,
        "books": books
    }

    return json.dumps(result, indent=2)


- Test the function by uncommenting the code below:

In [ ]:
# Example usage:
# print(search_books("machine learning", limit=5))

In [4]:
results = search_books("science fiction")
print(results)

📘 search_books called with query='science fiction' and limit=10
🔎 Sending request to Open Library API...
✅ Request successful, processing response...
📄 Found 99756 total results.
📚 Returning top 10 book results.

{
  "num_found": 99756,
  "books": [
    {
      "title": "The Science Fiction Hall of Fame -- Volume One",
      "authors": [
        "Robert Silverberg",
        "Robert A. Heinlein",
        "Arthur C. Clarke",
        "Isaac Asimov",
        "Ray Bradbury"
      ],
      "first_publish_year": 1970,
      "key": "/works/OL1960472W",
      "edition_count": 31
    },
    {
      "title": "Foundation",
      "authors": [
        "Isaac Asimov"
      ],
      "first_publish_year": 1951,
      "key": "/works/OL46125W",
      "edition_count": 97
    },
    {
      "title": "Stranger in a Strange Land",
      "authors": [
        "Robert A. Heinlein"
      ],
      "first_publish_year": 1961,
      "key": "/works/OL59688W",
      "edition_count": 80
    },
    {
      "title": "Th

## Step 3: Build a Chat Handler with Automatic Function Calling

- Drag and drop the raw cell below into the chat window and use a prompt like this:
  > Use this aisuite example to create a simple chat handler function `chat_with_tools`:
  > - input: `user_message`
  > - output: `response.choices[0].message.content`

In [5]:
import aisuite as ai

def chat_with_tools(user_message: str) -> str:
    """
    A simple chat handler that uses AISuite to interact with the model
    and automatically call available tools such as `search_books`.

    Parameters
    ----------
    user_message : str
        The user input message or query.

    Returns
    -------
    str
        The assistant's response content from the model.
    """
    # Initialize AISuite client
    client = ai.Client()

    # Prepare chat history with a single user message
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]

    # Send to model with tool access
    print("🤖 Chat handler activated: sending message to model...")
    response = client.chat.completions.create(
        model="openai:gpt-4.1-mini",
        messages=messages,
        tools=[search_books],
        max_turns=3  # allow up to 3 back-and-forth tool calls
    )

    # Extract and return the assistant's message content
    return response.choices[0].message.content


In [7]:
# Example usage:
reply = chat_with_tools("Find books about Artificial Intelligence")
print(reply)

🤖 Chat handler activated: sending message to model...
📘 search_books called with query='Artificial Intelligence' and limit=5
🔎 Sending request to Open Library API...
✅ Request successful, processing response...
📄 Found 21509 total results.
📚 Returning top 5 book results.

Here are some books about Artificial Intelligence:

1. "Artificial Intelligence" by Stuart J. Russell and Peter Norvig, first published in 1994.
2. "Artificial Intelligence" by Patrick Henry Winston, first published in 1977.
3. "Artificial Intelligence" by Elaine A. Rich, first published in 1983.
4. "Artificial Intelligence and Intelligent Systems" by N. P. Padhy, first published in 2005.
5. "Artificial Intelligence and Soft Computing" by Leszek Rutkowski, first published in 2012.

If you want more information about any of these books or other books on the topic, feel free to ask!


## Step 4: Test the Chat Handler

- Run the following code cell to test `chat_with_tools`
- Feel free to search for different types of books
- Optional: If your generated function `search_books` does not contain printing statements, you can update the function as Andrew showed in the video:
  - prompt used: modify the search_books function to add a printing statement showing when it is called with basic status messages.

In [6]:
print("="*60)
print("EXAMPLE: Science Function book search")
print("="*60)

user_query = "Search for books about science fiction"
print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")

EXAMPLE: Science Function book search

👤 User: Search for books about science fiction
🤖 Chat handler activated: sending message to model...
📘 search_books called with query='science fiction' and limit=5
🔎 Sending request to Open Library API...
✅ Request successful, processing response...
📄 Found 99756 total results.
📚 Returning top 5 book results.


🤖 Assistant: Here are some books about science fiction:

1. The Science Fiction Hall of Fame -- Volume One by Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury (First published in 1970)
2. Foundation by Isaac Asimov (First published in 1951)
3. Stranger in a Strange Land by Robert A. Heinlein (First published in 1961)
4. The Last Man by Mary Shelley (First published in 1826)
5. The Time Machine by H. G. Wells (First published in 1895)

Would you like more information about any of these books?


## Step 5: Look up Details on Books (optional) 

You can repeat steps 1-3 to define a second tool for the book research assistant, so it can handle more complex book queries.
For example, you can use the `works` endpoint to retrieve more detailed information about a particular book given its work id. For more details, check `openlibrary_api_docs.md`.

Here are steps with suggested prompts:

1. In the chat interface, attach the file containing the API documentation of Open Library (`openlibrary_api_docs.md`) and use a prompt like this:
   > Write a code cell that uses Open Library's works endpoint to retrieve information about a book, based on the work's key that has this format: "/works/{work_id}". The result should include: title, description, subjects (limit to 10), first_sentence, covers (limit to 3). Print the returned results. Use the documentation for Open Library's API provided above.
2. In the chat interface, create the function tool using a prompt like this:
   > Wrap the code above in a function `get_book_details`:
   > - Function argument: `work_key` in this format: `/works/{work_id}`
   > - Output: JSON string with: title, description, subjects (limit to 10), first_sentence, covers (limit to 3)
   >  - Write good docstring
3. Update the `chat_with_tools` function to include the second tool as follows:
   
   ``` python
   response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books, get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
   )
   ```
   
4. Try this query:
   ```python
   user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
   ```

In [8]:
import requests
import json

def get_book_details(work_key: str) -> str:
    """
    Retrieve detailed information about a book from the Open Library Works API.

    This function queries the Open Library Works endpoint using the provided 
    work key (in the format "/works/{work_id}") and extracts relevant book details.
    It includes logging-like print statements to show progress and status.

    Parameters
    ----------
    work_key : str
        The Open Library work key identifying the book (e.g., "/works/OL14933374W").

    Returns
    -------
    str
        A JSON-formatted string containing:
        - title : The title of the work
        - description : The book's description or 'No description available'
        - subjects : Up to 10 subject tags from the work
        - first_sentence : The first sentence text (if available)
        - covers : Up to 3 cover IDs
    """
    print("📘 get_book_details called...")
    print(f"🔑 Work key received: {work_key}")

    base_url = "https://openlibrary.org"
    url = f"{base_url}{work_key}.json"
    print(f"🌐 Fetching data from: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()
        print("✅ Request successful! Data retrieved from Open Library.")
    except requests.exceptions.RequestException as e:
        error_msg = f"❌ Failed to retrieve data: {e}"
        print(error_msg)
        return json.dumps({"error": error_msg}, indent=4)

    data = response.json()

    print("🧩 Extracting relevant book details...")
    result = {
        "title": data.get("title", "No title available"),
        "description": (
            data["description"]["value"]
            if isinstance(data.get("description"), dict)
            else data.get("description", "No description available")
        ),
        "subjects": data.get("subjects", [])[:10],
        "first_sentence": (
            data["first_sentence"]["value"]
            if isinstance(data.get("first_sentence"), dict)
            else data.get("first_sentence", "N/A")
        ),
        "covers": data.get("covers", [])[:3]
    }

    print("📦 Extraction complete! Returning JSON-formatted details.")
    return json.dumps(result, indent=4)
    

In [9]:
# Example usage:
print(get_book_details("/works/OL14933374W"))

📘 get_book_details called...
🔑 Work key received: /works/OL14933374W
🌐 Fetching data from: https://openlibrary.org/works/OL14933374W.json
✅ Request successful! Data retrieved from Open Library.
🧩 Extracting relevant book details...
📦 Extraction complete! Returning JSON-formatted details.
{
    "title": "An English forest",
    "description": "No description available",
    "subjects": [],
    "first_sentence": "N/A",
    "covers": [
        5688862,
        7189461
    ]
}


In [10]:
import aisuite as ai

def chat_with_tools(user_message: str) -> str:
    """
    A simple chat handler that uses AISuite to interact with the model
    and automatically call available tools such as `search_books`.

    Parameters
    ----------
    user_message : str
        The user input message or query.

    Returns
    -------
    str
        The assistant's response content from the model.
    """
    # Initialize AISuite client
    client = ai.Client()

    # Prepare chat history with a single user message
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]

    # Send to model with tool access
    print("🤖 Chat handler activated: sending message to model...")
    response = client.chat.completions.create(
        model="openai:gpt-4.1-mini",
        messages=messages,
        tools=[search_books, get_book_details],
        max_turns=3  # allow up to 3 back-and-forth tool calls
    )

    # Extract and return the assistant's message content
    return response.choices[0].message.content


In [12]:
# Example usage
user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
results = chat_with_tools(user_query)
print()
print(results)

🤖 Chat handler activated: sending message to model...
📘 search_books called with query='title:Linear Algebra' and limit=10
🔎 Sending request to Open Library API...
✅ Request successful, processing response...
📄 Found 2910 total results.
📚 Returning top 10 book results.

📘 get_book_details called...
🔑 Work key received: /works/OL25400954W
🌐 Fetching data from: https://openlibrary.org/works/OL25400954W.json
✅ Request successful! Data retrieved from Open Library.
🧩 Extracting relevant book details...
📦 Extraction complete! Returning JSON-formatted details.

The book with the most editions that has 'Linear Algebra' in the title is "Bundle" by Jeffrey D. Camm, James J. Cochran, Michael J. Fry, Jeffrey W. Ohlmann, and David Ray Anderson. It has 97 editions. However, there is no specific description available for this book.

If you want, I can provide details about other notable books on linear algebra with many editions as well.
